In [1]:
import pandas as pd
import numpy as np
from tools_utils import data_preparation, runtime_tools, data_loading, feature_generation, feature_selection

In [2]:
config = runtime_tools.GetConfig().get_config()
columns_config = config['columns']
name_col = columns_config['name_col']
symbol_col = columns_config['symbol_col']
fromdate_col = columns_config['fromdate_col']
todate_col = columns_config['todate_col']

In [3]:
portfolio = runtime_tools.PortfolioReader.read_portfolio()
portfolio.head()

,name,symbol,start_date,end_date
0,apple,AAPL,2021-01-01,2025-10-01
1,google,GOOGL,2021-01-01,2025-10-01
2,microsoft,MSFT,2021-01-01,2025-10-01
3,amazon,AMZN,2021-01-01,2025-10-01
4,tesla,TSLA,2021-01-01,2025-10-01


In [4]:
data = data_loading.DataImporter().import_data(portfolio_df=portfolio)

In [5]:
data_apple = data[data[symbol_col] == 'AAPL']
data_apple.head()

,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,AAPL,2025-10-01,255.04,258.79,254.93,255.45,48713940
1,AAPL,2025-09-30,254.86,255.92,253.11,254.63,37704300
2,AAPL,2025-09-29,254.56,255.00,253.01,254.43,40127700
3,AAPL,2025-09-26,254.10,257.60,253.78,255.46,46076300
4,AAPL,2025-09-25,253.21,257.17,251.71,256.87,55202100


In [6]:
dataprep = data_preparation.DataPrep(run_date=config['run_date'])
data_apple = dataprep.convert_timestamp(data_apple, time_col=columns_config['date_col'])
train, test = dataprep.split_train_test(data_apple)

sufficient data exists past run date, splitting data.


Generate features

In [7]:
feature_generator = feature_generation.Indicator(df=data_apple, td=False)
apple_feature_data = feature_generator.generate_all_indicators()
apple_feature_data.head(20)

,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,sma_5,wma_5,momentum_7,k_14,d_3,rsi_14,r_14,ad
0,AAPL,2025-10-01,255.04,258.79,254.93,255.45,48713940,255.450000,255.450000,255.45,0.000000,0.000000,0.000000,0.000000,-3.558894e+07
1,AAPL,2025-09-30,254.86,255.92,253.11,254.63,37704300,255.040000,254.903333,254.63,0.000000,0.000000,0.000000,0.000000,-3.250282e+07
2,AAPL,2025-09-29,254.56,255.00,253.01,254.43,40127700,254.836667,254.666667,254.43,0.000000,0.000000,0.000000,0.000000,-1.536285e+07
3,AAPL,2025-09-26,254.10,257.60,253.78,255.46,46076300,254.992500,254.984000,255.46,0.000000,0.000000,50.243902,0.000000,-2.091131e+07
4,AAPL,2025-09-25,253.21,257.17,251.71,256.87,55202100,255.368000,255.612667,256.87,0.000000,0.000000,70.520231,0.000000,2.822463e+07
5,AAPL,2025-09-24,255.22,255.74,251.04,252.31,42303710,254.740000,254.593333,252.31,0.000000,0.000000,30.423940,0.000000,8.782924e+06
6,AAPL,2025-09-23,255.88,257.34,253.58,254.43,60275200,254.700000,254.490000,254.43,43.741935,0.000000,44.970414,56.258065,-2.424019e+07
7,AAPL,2025-09-22,248.30,256.64,248.12,256.08,105517416,255.030000,254.950000,0.63,74.601687,0.000000,52.671756,25.398313,6.740639e+07
8,AAPL,2025-09-19,241.23,246.30,240.21,245.50,163741314,253.038000,251.773333,-9.13,28.471475,48.938366,27.760393,71.528525,1.881286e+08
9,AAPL,2025-09-18,239.97,241.20,236.65,237.88,44249600,249.240000,246.720667,-16.55,5.555556,36.209572,20.706902,94.444444,1.678030e+08


In [8]:
apple_feature_data_pca = feature_selection.PCAFeatureSelector().transform(apple_feature_data)
apple_feature_data_pca.head(20)

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,DATE,SYMBOL,CLOSE
0,6.209666,-0.830259,12.210560,-0.085030,-0.152612,-0.341367,0.508209,0.205415,2025-10-01,AAPL,255.45
1,6.217152,-0.856509,12.153857,-0.455211,-0.120856,-0.368753,0.511833,0.232886,2025-09-30,AAPL,254.63
2,6.173869,-0.848306,12.151507,-0.382841,-0.128040,-0.364501,0.511525,0.237219,2025-09-29,AAPL,254.43
3,6.359696,0.479959,11.694851,-0.312531,0.466387,1.900762,-0.298286,0.272153,2025-09-26,AAPL,255.46
4,6.333346,1.047656,11.572787,-0.081593,0.694043,2.837109,-0.612821,0.329224,2025-09-25,AAPL,256.87
5,6.244544,-0.062107,11.751325,-0.385122,0.244177,0.982804,0.010106,0.271267,2025-09-24,AAPL,252.31
6,6.181062,0.166651,11.370058,0.176215,0.374467,1.755550,-0.132894,-0.684243,2025-09-23,AAPL,254.43
7,4.706642,-0.305876,0.002222,2.298370,-0.140111,-0.339774,-2.081967,0.384748,2025-09-22,AAPL,256.08
8,3.651530,-1.507351,-0.360131,4.190469,-0.251702,-0.279232,0.689467,0.136085,2025-09-19,AAPL,245.50
9,4.103536,-3.012864,-0.788062,0.366014,0.346001,-0.369735,0.841522,0.082635,2025-09-18,AAPL,237.88
